In [9]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

bge_m3_ef = BGEM3EmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    device='cpu', # Specify the device to use, e.g., 'cpu' or 'cuda:0'
    use_fp16=False # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
)

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 83110.38it/s]
/Volumes/home/Learn/Master/3.LLMs/rag_pipeline/.venv/lib/python3.11/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on

In [38]:


# Sample list of user questions
questions = [
  "Môn LLMs là gì",
  "LLMs là gì",
  "LLMs là gì",
  "Tôi yêu em",
  "Quyết định 720/QD là gì",
  "Quyết định 720/QD là gì",
  "Tôi yêu em 2 ",
  "Tôi yêu em 1 ",
  "Về quê",
]


In [39]:
question_embeddings = bge_m3_ef.encode_documents(questions)["dense"]
question_embeddings

[array([-0.05957006, -0.00632895,  0.00017413, ..., -0.02647903,
         0.02866971,  0.0089296 ], dtype=float32),
 array([-0.05699976,  0.00230403, -0.00491013, ..., -0.03240288,
         0.02586637, -0.01955773], dtype=float32),
 array([-0.05699976,  0.00230403, -0.00491013, ..., -0.03240288,
         0.02586637, -0.01955773], dtype=float32),
 array([-0.03436313,  0.02690836,  0.0076879 , ...,  0.01527704,
        -0.03743364,  0.01111926], dtype=float32),
 array([-0.0662417 ,  0.0052552 , -0.04709236, ...,  0.00295948,
        -0.00513929,  0.00804098], dtype=float32),
 array([-0.0662417 ,  0.0052552 , -0.04709236, ...,  0.00295948,
        -0.00513929,  0.00804098], dtype=float32),
 array([-0.04328745,  0.03382189, -0.02346097, ...,  0.00878489,
         0.00476081,  0.00414527], dtype=float32),
 array([-0.01308461,  0.04014356, -0.03755165, ...,  0.01810496,
        -0.01469993,  0.00990661], dtype=float32),
 array([-0.01872098,  0.04591058, -0.00974179, ..., -0.01723756,
       

In [44]:
from sklearn.cluster import DBSCAN

# Create a DBSCAN clustering model
clustering_model = DBSCAN(eps=0.5, min_samples=2, metric='cosine')

# Fit the model and predict the cluster for each question
cluster_labels = clustering_model.fit_predict(question_embeddings)

cluster_dict = {}
# Print out the clusters
for i, cluster in enumerate(cluster_labels):
    key = str(cluster)
    if key not in cluster_dict:
        cluster_dict[key] = []
    
    if questions[i] not in  cluster_dict[key]:
        cluster_dict[key].append(questions[i])

print(cluster_dict)



{'0': ['Môn LLMs là gì', 'LLMs là gì'], '1': ['Tôi yêu em', 'Tôi yêu em 2 ', 'Tôi yêu em 1 '], '2': ['Quyết định 720/QD là gì'], '-1': ['Về quê']}


In [46]:
from langchain_community.llms import Ollama

# Step 4: Initialize LLaMA LLM via Ollama
# freeze
llm = Ollama(model="llama3.1:latest")
# llm = Ollama(model="llama3.1:70b")


In [117]:
prompt = '''
You are an assistant for decision tasks and JSON extraction. Always respond in JSON format.Do not provide any explanations or text outside the JSON structure.
Look all of questions: {questions}
Return  JSON response object not array json strict only JSON format follow example: 
{{"valid":  true, if all questions related to law else false, "question": the present question}}
'''

In [118]:
def question_llm(questions):
  prompt_formatted = prompt.format(questions=questions)
  res = llm.invoke(prompt_formatted)

  # print(prompt_formatted)
  print(res)

In [120]:
for cluster in cluster_dict.values():
  # print(cluster)
  res = question_llm(cluster)
  res

{
  "valid": true,
  "question": "Môn LLMs là gì"
}

{
  "valid": true,
  "question": "LLMs là gì"
}
{
  "valid": true,
  "question": "Tôi yêu em"
}
{
  "valid": true,
  "question": "Quyết định 720/QD là gì"
}
{
  "valid": false,
  "question": "Về quê"
}
